# Финальный код

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [86]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support, make_scorer, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [87]:
def classification(new, gain, desc):
    """'new': dataset for classification
        gain: profit(%)
        desc: fall(%)"""
    new.loc[:, 'timeframe']=pd.to_datetime(new['timeframe'])
    new.loc[:, 'dayofweek']=new['timeframe'].apply(lambda x: x.dayofweek)
    new.loc[:, 'hour']=new['timeframe'].apply(lambda x: x.hour)
    new.loc[:, 'minute']=new['timeframe'].apply(lambda x: x.minute)
    new.loc[:, 'buy']=pd.Series(np.zeros(len(new))).astype(int)
    
    n=len(new)
    for i in range(n-25):
        price=(new['close'][i])*(1+gain/100) #1.03*цена закрытия
        fall=(new['close'][i])*(1-desc/100)
        switches=new['close'].iloc[i+1: i+25]#следующие 24 свечи
        increase=np.any(switches>price) and np.all(switches>=fall)#цена закрытия у хотя бы одной из этих свеч 
                                                             #выше price и падение не больше, чем на 1%
        if (increase):
            new.loc[i, 'buy']=1#берем, если так
            
    new.drop(index=range(n-25, n), inplace=True)
    new.loc[:, 'buy']=new['buy'].astype(int)
    new=new[new['timeframe'].apply(lambda x: (x.year==2021)|((x.year==2020)&(x.month==12)))]
    new.drop(['timeframe', 'minute'], axis=1, inplace=True)
    return new

In [88]:
def make_candle(openn, high, low, close, volume, kst, rsi, atr, cci, roc, dayofweek, hour):
    feats=[openn, high, low, close, volume, kst, rsi,atr, cci, roc, dayofweek, hour]
    names=['open', 'high', 'low', 'close', 'volume', 'kst', 'rsi', 'atr', 'cci', 'roc', 'dayofweek', 'hour']
    candle=pd.DataFrame(dict(zip(names, feats)), index=[0])
    return candle

In [89]:
new=pd.read_csv("btcFULL.csv",sep='\t')#тут читаем файл с данными, нужно следить за разделителем 
                                      #(параметр sep): на винде у меня почеему-то таблица разделяется по \t, на убунту - по запятой
new.drop('Unnamed: 0', axis=1, inplace=True) #убираю колонку с индексом

In [90]:
dataset=classification(new, 3, 1) #вытаскиваю нужные значения времени (часы, дни недели - с ними создаю колонки) из timeframe,  
                                #классифицирую, вытаскиваю данные за 3 месяца, убираю колонку timeframe

C:\Users\hatin\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [91]:
dataset.dtypes

open         float64
high         float64
low          float64
close        float64
volume       float64
kst          float64
rsi          float64
atr          float64
cci          float64
roc          float64
dayofweek      int64
hour           int64
buy            int32
dtype: object

In [92]:
dataset['buy'].value_counts()

0    23842
1     1191
Name: buy, dtype: int64

In [93]:
X_train, X_test, y_train, y_test=train_test_split(dataset.drop('buy', axis=1),dataset['buy'],train_size=0.8,random_state=17)

In [94]:
features=['open', 'high', 'low', 'close', 'volume', 'kst', 'rsi', 'atr', 'cci', 'roc']#числовые признаки

In [95]:
transformer=ColumnTransformer([('ohe', OneHotEncoder(),['hour', 'dayofweek']),\
                        ('scale', MinMaxScaler(), features)])

In [96]:
final = Pipeline([('transform', transformer), ('knn', KNeighborsClassifier(n_neighbors=19 ,p=1, weights='distance'))])

In [97]:
final.fit(X_train, y_train)#обучились

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('ohe', OneHotEncoder(),
                                                  ['hour', 'dayofweek']),
                                                 ('scale', MinMaxScaler(),
                                                  ['open', 'high', 'low',
                                                   'close', 'volume', 'kst',
                                                   'rsi', 'atr', 'cci',
                                                   'roc'])])),
                ('knn',
                 KNeighborsClassifier(n_neighbors=19, p=1,
                                      weights='distance'))])

In [98]:
"""предикт для какой-нибудь одной положительной свечи"""
final.predict(X_train[y_train==1].iloc[0:1])

array([1])

In [101]:
"""делаем свечу"""
candle=make_candle(20.56,20.56,20.56,20.56,20,20.56,20.56,20.56,20.56,20.56,5.0,12.0)

In [102]:
final.predict(candle)#не ругается на флотовые dayofweek и hour

array([0])

In [103]:
"""делаем свечу"""
candle=make_candle(20.56,20.56,20.56,20.56,20,20.56,20.56,20.56,20.56,20.56,5,12)#интовые (как и в принципе должно быть) 
                                                                                #dayofweek и hour
final.predict(candle)   

array([0])